Create **Job Hiring Post** based on company description, comapny domain, hiring needs, and offered benefits(eg: remote work, USD salary)

In [ ]:
! pip install -q "groq==0.13.0" "crewai==0.86.0" crewai-tools "poetry==1.8.5" pydantic python-dotenv

In [2]:
from typing import List
from crewai import Agent, Crew, Process, Task, LLM
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool, FileReadTool
from pydantic import BaseModel, Field
import os
import sys

In [3]:
os.environ["OPENAI_API_KEY"] = "your-api-key"

In [4]:
os.environ["SERPER_API_KEY"] = "your-api-key"

In [5]:
model = "gpt-3.5-turbo"

In [6]:
web_search_tool = WebsiteSearchTool()
seper_dev_tool = SerperDevTool()

# read the example job posting file
file_read_tool = FileReadTool(
    file_path='/content/job_description_example.md',
    description='A tool to read the job description example file.'
)

##### Agents

In [7]:
research_agent = Agent(
    role="Research Analyst",
    goal= "Analyze the company website and provided description to extract insights on culture, values, and specific needs.",
    backstory= """
    Expert in analyzing company cultures and identifying key values
    and needs from various sources, including websites and brief descriptions. """,
    tools=[web_search_tool, seper_dev_tool],
    verbose=True
)

writer_agent = Agent(
    role= "Job Description Writer",
    goal= "Use insights from the Research Analyst to create a detailed, engaging, and enticing job posting.",
    backstory= """Skilled in crafting compelling job descriptions that resonate
    with the company's values and attract the right candidates.""",
    tools=[web_search_tool, seper_dev_tool, file_read_tool],
    verbose=True
)

review_agent = Agent(
    role= "Review and Editing Specialist",
    goal= "Review the job posting for clarity, engagement, grammatical accuracy, and alignment with company values and refine it to ensure perfection.",
    backstory= """A meticulous editor with an eye for detail, ensuring every piece of content is clear, engaging, and grammatically perfect.""",
    tools=[web_search_tool, seper_dev_tool, file_read_tool],
    verbose=True
)

##### Tasks

In [8]:
# Analyze the company based on company domain and description
research_company_culture_task = Task(
    description = """Analyze the provided company website and the hiring manager's company's domain {company_domain},
    description {company_description}. Focus on understanding the company's culture, values, and mission.
    Identify unique selling points and specific projects or achievements highlighted on the site.
    Compile a report summarizing these insights, specifically how they can be leveraged in a job posting
    to attract the right candidates.""",
    expected_output = """A comprehensive report detailing the company's culture, values, and mission, along with specific selling
    points relevant to the job role. Suggestions on incorporating these insights into the job posting should be included.""",
    agent = research_agent,
)

# Analyze the role requirements
research_role_requirements_task = Task(
    description= """Based on the hiring manager's needs: {hiring_needs}, identify the key skills, experiences,
    and qualities the ideal candidate should possess for the role. Consider the company's current projects,
    its competitive landscape, and industry trends. Prepare a list of recommended job requirements
    and qualifications that align with the company's needs and values.""",
    expected_output= """A list of recommended skills, experiences, and qualities for the ideal candidate, aligned with
    the company's culture, ongoing projects, and the specific role's requirements.""",
    agent = research_agent,
    output_file="job_posting.md"
)

# Create the draft for job hiring post using writer agent
draft_job_posting_task = Task(
    description= """Draft a job posting for the role described by the hiring manager: {hiring_needs}.
    Use the insights on {company_description} to start with a compelling introduction,
    followed by a detailed role description, responsibilities, and required skills and qualifications.
    Ensure the tone aligns with the company's culture and incorporate any unique benefits or
    opportunities offered by the company. Specific benefits: {specific_benefits}.""",
    expected_output= """A detailed, engaging job posting that includes an introduction, role description, responsibilities,
    requirements, and unique company benefits. The tone should resonate with the company's culture
    and values, aimed at attracting the right candidates.""",
    agent = writer_agent,
)

# Review the draft for the created job posting - Final Job hiring Post
review_and_edit_job_posting_task = Task(
    description= """Review the draft job posting for the role {hiring_needs}. Check for clarity, engagement, grammatical accuracy,
    and alignment with the company's culture and values. Edit and refine the content, ensuring it speaks directly
    to the desired candidates and accurately reflects the role's unique benefits and opportunities. Provide
    feedback for any necessary revisions.""",
    expected_output=  """A polished, error-free job posting that is clear, engaging, and perfectly aligned with the company's culture and values.
    Feedback on potential improvements and final approval for publishing. Formatted in markdown.""",
    agent = review_agent,
)

industry_analysis_task = Task(
    description= """Conduct an in-depth analysis of the industry related to the company's domain {company_domain}.
    Investigate current trends, challenges, and opportunities within the industry, utilizing market reports,
    recent developments, and expert opinions. Assess how these factors could impact the role being hired
    for and the overall attractiveness of the position to potential candidates.
    Consider how the company's position within this industry and its response to these trends could be leveraged to attract top talent.
    Include in your report how the role contributes to addressing industry challenges or seizing opportunities.""",
    expected_output=  """A detailed analysis report that identifies major industry trends, challenges, and opportunities relevant
    to the company's domain and the specific job role. This report should provide strategic insights on positioning
    the job role and the company as an attractive choice for potential candidates.""",
    agent = research_agent,
)


##### Crew

In [9]:
crew = Crew(
    agents=[research_agent, writer_agent, review_agent],
    tasks=[research_company_culture_task, research_role_requirements_task, draft_job_posting_task, review_and_edit_job_posting_task, industry_analysis_task],
    process=Process.sequential,
    verbose=True,
)

In [13]:
inputs = {
    'company_domain':'mitrai.com',
    'company_description': "At Mitra, we’re deeply committed to our people, their well-being, and their careers. This passion has driven us to launch Experienz, a new ESG monitoring and reporting initiative that enhances the employee experience. Our dedication extends to making a global impact by supporting eco-conscious entrepreneurs through Mitra Ventures. We help them create prototypes, scale up sustainably, and put ESG at the core of their propositions, all while caring for their businesses and products",
    'hiring_needs': 'AI enginner',
    'specific_benefits':'remote work, USD salary',
}

crew.kickoff(inputs=inputs)

# Agent: Research Analyst
## Task: Analyze the provided company website and the hiring manager's company's domain mitrai.com,
    description At Mitra, we’re deeply committed to our people, their well-being, and their careers. This passion has driven us to launch Experienz, a new ESG monitoring and reporting initiative that enhances the employee experience. Our dedication extends to making a global impact by supporting eco-conscious entrepreneurs through Mitra Ventures. We help them create prototypes, scale up sustainably, and put ESG at the core of their propositions, all while caring for their businesses and products. Focus on understanding the company's culture, values, and mission.
    Identify unique selling points and specific projects or achievements highlighted on the site.
    Compile a report summarizing these insights, specifically how they can be leveraged in a job posting
    to attract the right candidates.


Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.47it/s]




# Agent: Research Analyst
## Thought: I need to gather information about the company culture, values, and mission of Mitra from their website to create a comprehensive report. I will start by searching their website to extract relevant insights.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"company culture values mission\", \"website\": \"https://mitrai.com\"}"
## Tool Output: 
Relevant Content:
& legacy application integration Data integration & migration Managed integration services WORKFLOW AUTOMATION AI-powered workflow automation No-code/low-code development Integration with existing systems ML modeling for business optimization Workflow monitoring and analytics Don't just take our word for it “The Mitra team showed us how we could realise our ambitions with a tailor-made integration platform” Vladimir Klein Chief Information Officer Moneta Money Bank Read case study “Mitra’s AI automation expertise transformed our testing, boosting efficiency

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.61it/s]




# Agent: Research Analyst
## Thought: Thought: I have gathered information on the necessary skills and qualifications for an AI engineer. I will now extract relevant insights about specific requirements that align with Mitra's culture, ongoing projects, and values.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"AI engineer qualifications Mitra\", \"website\": \"https://www.mitra.com\"}"
## Tool Output: 
Relevant Content:
& legacy application integration Data integration & migration Managed integration services WORKFLOW AUTOMATION AI-powered workflow automation No-code/low-code development Integration with existing systems ML modeling for business optimization Workflow monitoring and analytics Don't just take our word for it “The Mitra team showed us how we could realise our ambitions with a tailor-made integration platform” Vladimir Klein Chief Information Officer Moneta Money Bank Read case study “Mitra’s AI automation expertise transformed our tes

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.69it/s]




# Agent: Job Description Writer
## Thought: I need to gather more information about Mitra's culture, values, and any unique benefits offered in order to craft an engaging and compelling job posting for the AI engineer role.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"Mitra company values and benefits\", \"website\": \"https://mitra.com\"}"
## Tool Output: 
Relevant Content:
& legacy application integration Data integration & migration Managed integration services WORKFLOW AUTOMATION AI-powered workflow automation No-code/low-code development Integration with existing systems ML modeling for business optimization Workflow monitoring and analytics Don't just take our word for it “The Mitra team showed us how we could realise our ambitions with a tailor-made integration platform” Vladimir Klein Chief Information Officer Moneta Money Bank Read case study “Mitra’s AI automation expertise transformed our testing, boosting efficiency and enabling innova

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.65it/s]




# Agent: Research Analyst
## Thought: Thought: I have gathered several relevant sources on current trends, challenges, and opportunities within the AI industry. Now, I will analyze the information from these sources to develop a comprehensive report that addresses the specific aspects of the AI Engineer role at Mitra and how it aligns with industry dynamics.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"Mitra AI Engineer role responsibilities trends challenges opportunities\", \"website\": \"mitrai.com\"}"
## Tool Output: 
Relevant Content:
Leading the way in Al-Powered digital cloud engineering Skip to main content Close Search Leading the way in Al-powered digital cloud engineering Revolutionize your applications with Al and data, modernizing cloud solutions to enhance customer experience Schedule a meeting AI Titans Clash: Who Will Dominate the Future of Generative AI READ BLOG ARTICLE Mitra AI framework achieves up to 40% productivity gain Lear

CrewOutput(raw="## Industry Analysis Report: AI Engineering Role at Mitra\n\n### Introduction\nThis report provides an in-depth analysis of the artificial intelligence (AI) industry as it relates to the role of an AI Engineer at Mitra, headquartered at mitrai.com. The report examines current trends, challenges, and opportunities within the AI sector, and assesses how these factors impact the position being hired for, along with the overall attractiveness of the role to potential candidates.\n\n### Industry Trends\n1. **Generative AI Boom**: 2023 has seen a significant surge in the adoption of generative AI technologies across various sectors. Companies are investing heavily in AI for content generation, automation, and innovation which drives demand for engineers proficient in AI technologies (McKinsey).\n\n2. **Increased Focus on Ethics and Governance**: As AI technologies continue to grow, ethical considerations surrounding data privacy, algorithm bias, and transparency are becoming 